In [9]:
import os
import numpy as np
import librosa

# 定义数据路径
base_path = r"D:\Dataset\Temp_Results\Train_Test"
output_path = r"D:\Dataset\Temp_Results\SNRandLHR"
os.makedirs(output_path, exist_ok=True)

# 读取音频数据和对应的采样率及 ID
X_train = np.load(os.path.join(base_path, "X_train.npy"), allow_pickle=True)
X_test = np.load(os.path.join(base_path, "X_test.npy"), allow_pickle=True)
sr_train = np.load(os.path.join(base_path, "sr_train.npy"), allow_pickle=True)
sr_test = np.load(os.path.join(base_path, "sr_test.npy"), allow_pickle=True)
ids_train = np.load(os.path.join(base_path, "ids_train.npy"), allow_pickle=True)
ids_test = np.load(os.path.join(base_path, "ids_test.npy"), allow_pickle=True)

def compute_snr_lhr_adaptive(signal, sample_rate):
    fft_signal = np.fft.fft(signal)
    psd_signal = np.abs(fft_signal) ** 2
    freqs = np.fft.fftfreq(len(signal), d=1/sample_rate)
    freqs = freqs[:len(freqs) // 2]
    psd_signal = psd_signal[:len(psd_signal) // 2]
    
    spectral_centroid = np.sum(freqs * psd_signal) / (np.sum(psd_signal) + 1e-8)
    geometric_mean = np.exp(np.mean(np.log(psd_signal + 1e-8)))
    arithmetic_mean = np.mean(psd_signal)
    sfm = geometric_mean / (arithmetic_mean + 1e-8)
    threshold = np.percentile(psd_signal, 90 * (1 - sfm))
    signal_power = np.sum(psd_signal[psd_signal > threshold])
    noise_power = np.sum(psd_signal[psd_signal <= threshold])
    snr = 10 * np.log10(signal_power / (noise_power + 1e-8))
    
    low_freq_power = np.sum(psd_signal[freqs < spectral_centroid])
    high_freq_power = np.sum(psd_signal[freqs >= spectral_centroid])
    lhr = low_freq_power / (high_freq_power + 1e-8)
    
    return snr, lhr

def process_audio_data(X, sr, ids, set_name):
    snr_lhr_list = []
    
    for i, signal in enumerate(X):
        snr, lhr = compute_snr_lhr_adaptive(signal, sr[i])
        snr_lhr_list.append([ids[i], snr, lhr])
    
    snr_lhr_array = np.array(snr_lhr_list, dtype=object)
    np.save(os.path.join(output_path, f"SNR_LHR_{set_name}.npy"), snr_lhr_array)
    print(f"✅ {set_name} 数据处理完成，结果已保存。")

# 处理训练和测试数据
process_audio_data(X_train, sr_train, ids_train, "train")
process_audio_data(X_test, sr_test, ids_test, "test")


✅ train 数据处理完成，结果已保存。
✅ test 数据处理完成，结果已保存。


In [10]:
X_train_SNR_LHR = np.load(r"D:\Dataset\Temp_Results\SNRandLHR\SNR_LHR_train.npy", allow_pickle=True)
print(X_train_SNR_LHR)


[[np.str_('5903700F_seg0') np.float32(25.175419) np.float32(14.649713)]
 [np.str_('59037013_seg7') np.float32(21.275179) np.float32(6.195104)]
 [np.str_('5903700G_seg6') np.float32(27.6208) np.float32(17.714106)]
 ...
 [np.str_('61072032_seg17') np.float32(15.2527485) np.float32(1.5869788)]
 [np.str_('6107200Z_seg1') np.float32(9.539703) np.float32(1.0238315)]
 [np.str_('59037007_seg10') np.float32(21.410053) np.float32(23.620064)]]
